In [288]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [289]:
df = pd.read_csv('star_classification.csv')
df.drop(['obj_ID'], axis = 1,inplace=True) #dropping OBJ_ID feature as it is just the ID
df.drop(['rerun_ID'], axis = 1,inplace=True)#dropping rerun_ID feature as it is the same for all
df.drop(['run_ID'], axis = 1,inplace=True)
df.drop(['field_ID'], axis = 1,inplace=True)
df.drop(['spec_obj_ID'], axis = 1,inplace=True)
df

,alpha,delta,u,g,r,i,z,cam_col,class,redshift,plate,MJD,fiber_ID
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,2,GALAXY,0.634794,5812,56354,171
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,5,GALAXY,0.779136,10445,58158,427
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,2,GALAXY,0.644195,4576,55592,299
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,3,GALAXY,0.932346,9149,58039,775
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,3,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,2,GALAXY,0.000000,9374,57749,438
99996,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,1,GALAXY,0.404895,7626,56934,866
99997,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,4,GALAXY,0.143366,2764,54535,74
99998,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,4,GALAXY,0.455040,6751,56368,470


In [290]:
data = np.array(df)

In [291]:
#dealing with class labels (changing class strings to 0 1 2):
def label_corrector(array):
    temp = list()
    seen_labels = set()
    dict_of_labels = dict()
    label_counter = -1
    for label in array:
        if label in seen_labels:
            temp.append(dict_of_labels[label])
        else:
            label_counter += 1
            dict_of_labels[label] = label_counter
            seen_labels.add(label)
            temp.append(label_counter)
    return np.array(temp)
            


In [292]:
class_index = 8
data[:,class_index] = label_corrector(data[:,class_index])
train_d = data[:80000]
train_x = np.concatenate((train_d[:,:class_index],train_d[:,class_index+1:]),axis=1)
train_y = train_d[:,class_index]

test_d = data[80000:]
test_x = np.concatenate((test_d[:,:class_index],test_d[:,class_index+1:]),axis=1)
test_y = test_d[:,class_index]

In [293]:
#getting the feature histograms
tdata = np.transpose(train_x) #transposed training matrix
histograms = list()
for col in  tdata:
    hst = np.histogram(col,bins = 100) 
    histograms.append(hst)
    #plt.plot(hst[1][1:],hst[0])
    #plt.show()

In [294]:
#histograms show that there are outliers, to be sure, IQR concept is used to look at outliers:
IQRs = list()
for ind, col in  enumerate(tdata):
    percentiles = np.percentile(col,[25,75]) #calclating 25 and 75 th percentiles
    IQR = percentiles[1]-percentiles[0] #calulating interquartile range
    right = col>(percentiles[1]+1.5*IQR)
    left = col <(percentiles[0]-1.5*IQR)
    outliers = right+left
    print(f"{ind}th feature has {sum(outliers)} outliers.")
    

0th feature has 0 outliers.
1th feature has 0 outliers.
2th feature has 44 outliers.
3th feature has 83 outliers.
4th feature has 109 outliers.
5th feature has 201 outliers.
6th feature has 267 outliers.
7th feature has 0 outliers.
8th feature has 7272 outliers.
9th feature has 0 outliers.
10th feature has 0 outliers.
11th feature has 0 outliers.


In [295]:
# Outliers can be removed given that it is not a critical task (serious applications) and there are not many outliers
#9th feature has many outliers, close to 5-10 percent. Hence, only the outliers of 2 3 4 5 6 th features will be removed.

#removing outliers:
IQRs = list()
outlier_rows = np.array([])
for ind, col in  enumerate(tdata):
    if ind in [3,4,5,6,2]:
        percentiles = np.percentile(col,[25,75])
        IQR = percentiles[1]-percentiles[0]
        right = (col>percentiles[1]+1.5*IQR)*1
        left = (col < (percentiles[0]-1.5*IQR))*1
        outliers = right+left
        #print(f"{ind}th feature has {sum(outliers)} outliers.")
        outlier_rows = np.append(outlier_rows,np.where(outliers == 1)[0])
outlier_rows = np.array(list(set(outlier_rows)))
outlier_rows = np.array(outlier_rows,dtype = 'int')
train_x = np.delete(train_x,outlier_rows,axis=0)
train_y = np.delete(train_y,outlier_rows,axis=0)

In [296]:
#Looking at histograms after outliers are removed:
#getting the feature histograms:
tdata = np.transpose(train_x) #transposed training matrix
histograms = list()
for col in  tdata:
    hst = np.histogram(col,bins = 100) 
    histograms.append(hst)
    #plt.plot(hst[1][1:],hst[0])
    #plt.show()

In [297]:
means_x = list()
stds_x = list()

for ind in range(np.shape(train_x)[1]):
    col = train_x[:,ind]
    mean = np.mean(col)
    std = np.std(col)
    means_x.append(mean)
    stds_x.append(std)
    train_x[:,ind] = (col-mean)/std
    

    
    

for ind in range(np.shape(test_x)[1]):
    col = test_x[:,ind]
    test_x[:,ind] = (col-means_x[ind])/stds_x[ind]

    


# Decision Tree Implementation

In [325]:
class Node:
    #The tree is defined with nodes that have nodes in them. The process will be implemented recursively 
    #in later parts.
    
    def __init__(self, depth=None,right_node=None, left_node=None,node_type=None,node_class = None,split_feature = None,split_threshold = None ):
        self.left_node = left_node
        self.right_node =right_node
        self.node_type = node_type
        self.node_class = node_class
        self.split_feature = split_feature
        self.split_threshold = split_threshold

class DecisionTree:
    def __init__(self, data=None, labels=None,max_depth =100,min_member=5):
        self.max_depth=max_depth
        self.data = data
        self.labels = labels
        self.min_member=min_member
    def build_the_tree(self,data,labels,current_depth):
        
        num_instances, num_features = np.shape(data)
        split_feature,split_threshold,gini = get_splitting_point(data,labels) #returns the feature index and the threshold
        
        if gini > 0 and current_depth< self.max_depth and len(labels) > self.min_member:
            right_data, left_data,right_labels,left_labels = get_split(split_feature, split_threshold, data,labels)
            right_node = self.build_the_tree(data=right_data,labels = right_labels,current_depth=current_depth+1)
            left_node = self.build_the_tree(data = left_data,labels=left_labels,current_depth=current_depth+1)
            return Node(node_type = "branch",right_node = right_node, left_node = left_node,split_feature=split_feature,split_threshold=split_threshold,depth=current_depth  )
        return Node(node_type = "leaf",node_class = self.get_class(data,labels))
    def get_splitting_point(self,data,labels):
        num_instances, num_features = np.shape(data)
        gini =np.inf
        for feature in range(num_features):
            split_threshold, gini_gain = best_of_feature_split(data,labels,feature)
            if gini_gain<gini:
                gini = gini_gain
                best_split_threshold , best_split_feature = split_threshold,feature
        return best_split_feature,best_split_threshold,gini 
        #return split_feature, split_threshold, gini_gain
    def best_of_feature_split(self,data,labels,split_feature):
        feature_col = data[:,split_feature]
        thresholds = np.unique(feature_col, return_counts=False) #using the values as thresholds to make it efficient
        min_gini = np.inf 
        for threshold in thresholds:
            right_data, left_data,right_labels,left_labels = get_split(split_feature,threshold,data,labels)
            len_right = len(right_labels)
            len_left = len(left_labels)
            gini = gini_index(right_data,right_labels)*(len_right/(len_right+len_left))+gini_index(left_data,left_labels)*(len_left/(len_right+len_left))
            if gini < min_gini:
                min_gini = gini
                split_threshold = threshold
        return split_threshold, min_gini
        
        
    def get_class(self,data,labels):
        unique, counts = np.unique(labels, return_counts=True)
        counter = dict(zip(unique, counts))
        return max(labels, key=lambda x: counter[x])
    def gini_index(self,data,labels):
        length = len(labels)
        unique, counts = np.unique(labels, return_counts=True)
        counter = dict(zip(unique, counts))
        gini = 1
        for val in counter.values():
            gini -= (val/length)**2
        return gini
    def get_split(self,split_feature,split_threshold,data,labels):
        right_labels=np.array([])
        left_labels=np.array([])
        right_data=np.array([])
        left_data=np.array([])
        num_instances, num_features = np.shape(data)
        right_count = 0

        for ind, row in enumerate(data):

            if row[split_feature]>split_threshold:
                right_data=np.append(right_data,row)
                right_labels=np.append(right_labels,labels[ind])
                right_count +=1
            else:
                left_data=np.append(left_data,row)
                left_labels=np.append(left_labels,labels[ind])
        right_data = right_data.reshape(right_count,num_features)
        left_data = left_data.reshape(num_instances-right_count,num_features)
        return right_data, left_data,right_labels,left_labels 
    def fit(self):
        self.decision_maker = self.build_the_tree(self.data,self.labels,0) #this returns a node instance 
    def predict(self,new_data):
        predictions = list()
        for row in new_data:
            predictions.append(self.infer(self.decision_maker,row))
        return predictions
            
    def infer(self,node,datapoint):
        if node.node_type == "branch":
            val = datapoint[node.split_feature] 
            if val > node.split_threshold:
                return self.infer(node.right_node,datapoint)
            else:
                return self.infer(node.left_node,datapoint)
        elif node.node_type == "leaf":
                return node.node_class
    


**Training and Inference Time**

In [334]:
Tree = DecisionTree(train_x,train_y)

In [ ]:
Tree.fit()

In [ ]:
predictions = Tree.predict(test_x)

In [ ]:
count = 0
for ind,prediction in enumerate(predictions):
    if test_y[ind]==prediction:
        count+=1
print((count/len(predictions))*100)

In [328]:


from sklearn import datasets


# import some data to play with
iris = datasets.load_iris()
X = iris.data  # we only take the first two features.
y = iris.target


In [329]:
from sklearn.model_selection import train_test_split
trainx, testx, trainy, testy = train_test_split(X, y, test_size=0.2, random_state=42)

In [330]:
Tree = DecisionTree(trainx,trainy)

In [331]:
Tree.fit()

In [332]:
predictions = Tree.predict(testx)

In [333]:
count = 0
for ind,prediction in enumerate(predictions):
    if testy[ind]==prediction:
        count+=1
print((count/len(predictions))*100)

100.0


# Evaluation Metrics